In [ ]:
import anndata
import pandas as pd
import numpy as np
import os
import sys

from grelu.resources import load_model

## Paths

In [ ]:
matrix_file='/gstore/data/resbioai/grelu/decima/20240823/data.h5ad'

out_dir='/gstore/data/resbioai/grelu/decima/20240823/gwas_44traits/positive_variants'

pos_file = os.path.join(out_dir, 'positive_variants_processed.csv')
decima_preds_file = os.path.join(out_dir, 'decima_preds.npy')
gene_preds_file = os.path.join(out_dir, 'gene_scores.npy')
tss_preds_file = os.path.join(out_dir, 'tss_scores.npy')

## Load data

In [ ]:
ad = anndata.read_h5ad(matrix_file)

## Load variants

In [ ]:
pos = pd.read_csv(pos_file)

## Load predictions

In [ ]:
decima_preds = np.load(decima_preds_file)
borzoi_gene_preds = np.load(gene_preds_file)
borzoi_tss_preds = np.load(tss_preds_file)
decima_preds.shape, borzoi_gene_preds.shape, borzoi_tss_preds.shape

## Load Borzoi targets

In [ ]:
model = load_model(project="borzoi", model_name=f"human_fold0")
tasks = pd.DataFrame(model.data_params['tasks'])
del model

## For Decima: average VEP per cell type

In [ ]:
idx_map = ad.obs.reset_index().groupby(['cell_type']).agg({'index': tuple}).reset_index().dropna()
idx_map.head()

In [ ]:
decima_preds = idx_map['index'].apply(lambda idxs:decima_preds[:, ad.obs_names.isin(idxs)].mean(1))
decima_preds = np.stack(decima_preds).T
decima_preds.shape

In [ ]:
gene_exp = idx_map['index'].apply(lambda idxs:ad.X[ad.obs_names.isin(idxs), :].mean(0))
gene_exp = np.stack(gene_exp).T
gene_exp.shape

In [ ]:
var = pd.DataFrame(index=idx_map.cell_type.astype(str))

## For Borzoi: subset to relevant tasks

In [ ]:
borzoi_gene_preds = borzoi_gene_preds[:, tasks.assay=='RNA']
borzoi_tss_preds = borzoi_tss_preds[:, tasks.assay=='CAGE']
borzoi_gene_preds.shape, borzoi_tss_preds.shape

## For all methods: choose the best gene (highest absmax VEP) per variant

In [ ]:
idx_map = pos.reset_index().groupby(['variant'])['index'].apply(list).reset_index()
idx_map.head()

In [ ]:
idx_map['best_decima'] = idx_map['index'].apply(
    lambda x: x[np.abs(decima_preds[x]).mean(1).argmax()])

In [ ]:
idx_map['best_borzoi_gene'] = idx_map['index'].apply(
    lambda x: x[np.abs(borzoi_gene_preds[x]).mean(1).argmax()])
idx_map['best_borzoi_tss'] = idx_map['index'].apply(
    lambda x: x[np.abs(borzoi_tss_preds[x]).mean(1).argmax()])

## Subset predictions to the matched genes

In [ ]:
decima_preds = anndata.AnnData(
    X=decima_preds[idx_map.best_decima], var=var, obs=pos.iloc[idx_map.best_decima].reset_index(drop=True))
decima_preds.shape

In [ ]:
gene_exp = gene_exp[decima_preds.obs.gene.apply(lambda x: np.where(ad.var_names==x)[0][0])]
gene_exp.shape

In [ ]:
decima_preds.layers['gene_exp'] = gene_exp

In [ ]:
borzoi_gene_preds = anndata.AnnData(
    X=borzoi_gene_preds[idx_map.best_borzoi_gene], var=tasks[tasks.assay=='RNA'],
    obs=pos.iloc[idx_map.best_borzoi_gene].reset_index(drop=True))
borzoi_gene_preds.shape

In [ ]:
borzoi_tss_preds = anndata.AnnData(
    X=borzoi_tss_preds[idx_map.best_borzoi_tss], var=tasks[tasks.assay=='CAGE'],
    obs=pos.iloc[idx_map.best_borzoi_tss].reset_index(drop=True))
borzoi_tss_preds.shape

## Save

In [ ]:
decima_out_file = os.path.join(out_dir, 'decima_preds_agg.h5ad')
gene_out_file = os.path.join(out_dir, 'gene_preds_agg.h5ad')
tss_out_file = os.path.join(out_dir, 'tss_preds_agg.h5ad')

In [ ]:
decima_preds.write_h5ad(decima_out_file)
borzoi_gene_preds.write_h5ad(gene_out_file)
borzoi_tss_preds.write_h5ad(tss_out_file)